# Optimized Model for Deployment

## Predict with Python (SLOW)

In [ ]:
from tensorflow.contrib import predictor
import numpy as np

saved_model_path = './linear_model/cpu/pipeline_tfserving/0'
print(saved_model_path)

input_shape = 1
input_data = np.random.random_sample(input_shape)

predict_fn = predictor.from_saved_model(saved_model_path)

In [ ]:
%%time
predictions = predict_fn({'inputs': input_data})

In [ ]:
print('Prediction: %s' % predictions["outputs"])

## Optimize with TensorFlow Lite

![PipelineAI + TensorFlow Lite](https://pipeline.ai/assets/img/toco-optimizer.png)

In [ ]:
import tensorflow as tf
import os

from shutil import rmtree

saved_model_path = './linear_model/cpu/pipeline_tfserving/0'
print(saved_model_path)

tflite_model_base_path = './linear_model/cpu/tflite/'

os.makedirs(tflite_model_base_path, exist_ok=True)

converter = tf.contrib.lite.TocoConverter.from_saved_model(saved_model_path)

# TF 1.11+
converter.post_training_quantize = True

tflite_model = converter.convert()

tflite_model_path = '%s/tflite_optimized_model.tflite' % tflite_model_base_path

model_size = open(tflite_model_path, "wb").write(tflite_model)

print('\nModel size reduced to %s bytes' % model_size)

In [ ]:
%%bash -s "$tflite_model_path"
echo "ls -al $1"
echo ""
ls -al $1

In [ ]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.contrib.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

In [ ]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
print('Input Tensor Details: %s' % input_details)

output_details = interpreter.get_output_details()
print('Output Tensor Details: %s' % output_details)

In [ ]:
# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print('Input: %s' % input_data)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [ ]:
%%time
interpreter.invoke()

In [ ]:
output_data = interpreter.get_tensor(output_details[0]['index'])
print('Prediction: %s' % output_data)